# ZenML: Open-source MLOps Framework for reproducible ML pipelines

![Test](../_assets/Logo/zenml.svg)

![Sam](../_assets/sam.png)

In [ ]:
from absl import logging as absl_logging
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
absl_logging.set_verbosity(-10000)

Let's begin by initializing ZenML in our directory. We are going to use a local stack to begin with, for simplicity and then transition to other stacks. This can be achieved in code by executing the following block.

# Initialize ZenML

In [ ]:
!zenml init

## The ZenML stack

The ZenML stack is a concept that describes the union of Metadata Store, Artifact Store and Orchestrator that will be used for all pipeline runs. When you get started with zenml you start off with a default local stack.

In [1]:
!zenml stack list

┏━━━━━━━━┯━━━━━━━━━━━┯━━━━━━━━━━━┯━━━━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━━┯━━━━━━━━━━┓
┃        │ STACK     │           │           │          │           │          ┃
┃ ACTIVE │ NAME      │ ORCHESTR… │ METADATA… │ ARTIFAC… │ CONTAINE… │ STEP_OP… ┃
┠────────┼───────────┼───────────┼───────────┼──────────┼───────────┼──────────┨
┃   👉   │ local_st… │ local_or… │ local_me… │ local_a… │           │          ┃
┗━━━━━━━━┷━━━━━━━━━━━┷━━━━━━━━━━━┷━━━━━━━━━━━┷━━━━━━━━━━┷━━━━━━━━━━━┷━━━━━━━━━━┛


### The Local Stack

You can imagine the local stack to look like this. Within the diagram we show how a generic pipeline interacts with the local stack.

![LocalStack](../_assets/localstack.png "LocalStack")

In [13]:
!zenml stack set local_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run.py

Active stack: local_stackto `local_stack`.....
⠇ Setting the active stack to `local_stack`...`local_stack`...
Creating run for pipeline: `continuous_deployment_pipeline`
Cache disabled for pipeline `continuous_deployment_pipeline`
Using stack `local_stack` to run pipeline `continuous_deployment_pipeline`...
Step `importer` has started.
Step `importer` has finished in 0.112s.
Step `get_reference_data` has started.
Step `get_reference_data` has finished in 0.094s.
Step `svc_trainer_mlflow` has started.
E0319 02:34:14.886944556   12498 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0319 02:34:14.899501217   12498 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0319 02:34:14.911337993   12498 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2022/03/19 02:34:15 WARNING mlflow.utils.autologging_utils: MLflow autologging enco

We will now use the Kubeflow integration to extend the concept of stacks

![All](../_assets/evidently+mlflow+discord+kubeflow.png "All")

In [14]:
!zenml integration install kubeflow -f

⠸ Installing integrations...Requirement already satisfied: kfp==1.8.9 in /home/htahir1/.virtualenvs/zenml_active/lib/python3.8/site-packages (1.8.9)
⠼ Installing integrations...Requirement already satisfied: pyrsistent>=0.14.0 in /home/htahir1/.virtualenvs/zenml_active/lib/python3.8/site-packages (from jsonschema<4,>=3.0.1->kfp==1.8.9) (0.18.1)
⠴ Installing integrations...Requirement already satisfied: pyasn1<0.5.0,>=0.4.6 in /home/htahir1/.virtualenvs/zenml_active/lib/python3.8/site-packages (from pyasn1-modules>=0.2.1->google-auth<2,>=1.6.1->kfp==1.8.9) (0.4.8)


⠼ Installing integrations...


### The Kubeflow Pipeline stack

Now we want to transition to a kubeflow stack that will look a little bit like this. Note that for kubeflow pipelines we also need a registry where the docker images for each step are registered. 

![KubeflowStack](../_assets/localstack-with-kubeflow-orchestrator.png "KubeflowStack")

But we have good news! You barely have to do anything to transition.

In [ ]:
# You register a container registry with zenml
!zenml container-registry register local_registry --type=default --uri=localhost:5000
    
# You register an orchestrator with zenml
!zenml orchestrator register kubeflow_orchestrator --type=kubeflow

# Now it all is combined into the local_kubeflow_stack
!zenml stack register local_kubeflow_stack \
    -m local_metadata_store \
    -a local_artifact_store \
    -o kubeflow_orchestrator \
    -c local_registry

# And we activate the new stack, now all pipelines will be run within this stack
!zenml stack set local_kubeflow_stack

# Check it out, your new stack is registered
!zenml stack list!zenml stack set local_kubeflow_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run_pipeline.py train

Registered stack component with name 'local_registry'.
Successfully registered container registry `local_registry`.


### Starting up your new kubeflow pipelines stack

All that is left to do is power up your stack. This is just one more line away. The stack up process might take some time for you. In the background k3d will be creating and starting up a cluster of docker containers to host kubeflow pipelines locally. 

In [ ]:
!zenml stack up

If you scroll down all the way on the previous output you should see a link to your running kubeflow pipelines UI. Most probably this will be at [http://localhost:8080/](http://localhost:8080/).

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently running pipelines defined within a jupyter notebook cell is
    not supported. To get around this you can run the train pipeline within this repo. 
</div>

In [ ]:
!zenml stack set local_kubeflow_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run_pipeline.py train